
# Benchmark Generator for External Version Explanation
This notebook generates synthetic datasets for evaluating systems that explain new attributes in updated tables using external joins.


In [5]:
# 1. Setup
import pandas as pd
import numpy as np
from pathlib import Path

# 2. Load CSVs
movies = pd.read_csv('data/movies.csv')  # movieId,title,genres
ratings = pd.read_csv('data/ratings.csv')  # userId,movieId,rating,timestamp
links = pd.read_csv('data/movies_links.csv')  # title,genres,imdbId,tmdbId
tmdb = pd.read_csv('data/movie_details.csv')  # extended metadata

In [6]:

# 3. Create output directory
output_dir = Path("generated_benchmark")
output_dir.mkdir(exist_ok=True)

# 4. Helper to save versions
def save_version(df, index, metadata):
    fname = output_dir / f"version_{index:02d}.csv"
    df.to_csv(fname, index=False)
    return {
        'version': f'version_{index:02d}.csv',
        'rows': len(df),
        'columns': list(df.columns),
        'source': metadata
    }

In [30]:

# 5. Generate joined tables (versions)
versions = []
np.random.seed(42)

# Version 1–5: movies + sampled ratings
for i in range(5):
    sampled = ratings.sample(frac=0.005)
    merged = pd.merge(movies, sampled, on='movieId', how='inner')
    undersampled = merged.sample(frac=np.random.uniform(0.1, 0.5))
    versions.append(save_version(undersampled, i, 'movies + sampled ratings [undersampled]'))

# Version 6–10: movies + links (random column subset)
for i in range(5):
    merged = pd.merge(movies, links, on=['title', 'genres'], how='left')
    reduced = merged.drop(columns=np.random.choice(merged.columns, size=2, replace=False))
    versions.append(save_version(undersampled, i + 5, 'movies + links - columns dropped [undersampled]'))

# Version 11–15: movies + tmdb (on imdbId, tmdbId) with filtering
linked = pd.merge(movies, links, on=['title', 'genres'], how='inner')
for i in range(5):
    enriched = pd.merge(linked, tmdb, on=['imdbId', 'tmdbId'], how='inner')
    filtered = enriched[enriched['popularity'] > enriched['popularity'].quantile(0.75)]
    undersampled = filtered.sample(frac=np.random.uniform(0.1, 0.5))
    versions.append(save_version(undersampled, i + 10, 'movies + links + tmdb [popular only, undersampled]'))

# Version 16–20: movies + tmdb + derived column
for i in range(5):
    enriched = pd.merge(linked, tmdb, on=['tmdbId'], how='outer')
    enriched['profit_ratio'] = enriched['revenue'] / (enriched['budget'] + 1).where(enriched['budget'] > 0,  enriched['revenue'])
    subset = enriched[['movieId', 'original_title', 'vote_average', 'tmdbId', 'popularity', 'profit_ratio']]
    subset = subset.dropna()
    subset = subset[subset['profit_ratio'] > 10]
    versions.append(save_version(subset, i + 15, 'movies + derived column [undersampled]'))

# Version 21–25: movies + average ratings (aggregated)
avg_ratings = ratings.groupby('movieId').agg(avg_rating=('rating', 'mean')).reset_index()
for i in range(5):
    merged = pd.merge(movies, avg_ratings, on='movieId', how='left')
    sampled = merged.sample(frac=np.random.uniform(0.1, 0.5))
    versions.append(save_version(sampled, i + 20, 'movies + avg rating [undersampled]'))

# Version 26–30: Simulated editing (row deletion, column renaming)
for i in range(5):
    sample = movies.sample(frac=np.random.uniform(0.1, 0.5))
    sample = sample.rename(columns={'title': 'film_title'})
    versions.append(save_version(sample, i + 25, 'movies subset with renamed column [undersampled]'))


In [11]:

# 6. Save benchmark metadata
pd.DataFrame(versions).to_json(output_dir / "benchmark_metadata.json", orient='records', indent=2)

print("✅ Benchmark generation complete: 30 versioned tables.")


✅ Benchmark generation complete: 30 versioned tables.
